In [ ]:
import os
import requests
import ollama
from bs4 import BeautifulSoup
from pypdf import PdfReader
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
###########################
### ChatGPT Credentials ###
###########################
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

################################
### Ollama Local Credentials ###
################################
OLLAMA_API = "http://localhost:11434/api/chat"
OLLAMA_HEADERS = {"Content-Type": "application/json"}
OLLAMA_MODEL = "llama3.2" #'tinyllama' #'phi3:mini'

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Class to store data about the uploaded Website
class Website:
    def __init__(self, url):
        self.url = url
        print('Scanning URL: ', url)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body("script", "style", "img", "input"):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        print('Successfully parsed Website')

# Class to store data about the uploaded Resume
class Resume:
    def __init__(self, resume_path):
        self.pdfReader = PdfReader(resume_path)
        self.text = self.pdfReader.pages[0].extract_text()
        print('Extracted Text from PDF!')

In [ ]:
system_prompt = "You are a resume checker and have internal knowledge about what the ATS (applicant tracking system) is identifying as needed when observing the uploaded resume"

In [ ]:
### Inform LLM what we want to happen to the Resume we're uploading.
###   - In this we explain that we are applying to the job listed on this Website & would like
###     the LLM to extract the title, text and resume data to give us feedback.
def user_prompt(website, resume):
    user_prompt = f"You are looking at a website titled: {website.title}, with this info: {website.text}"
    user_prompt += "\nPlease use this website to make key adjustments to the uploaded resume on hand. \
    I am the person looking to be hired by this company and would like to be given at least a fair chance to interview\
    Please suggest necessary changes and improvements in list format -- especially within the Projects or Experience sections.\
    Also, please make your suggestions using Markdown.\n\n" 
    user_prompt += resume.text
    return user_prompt
    

In [ ]:
### Create boilerplate array of dictionaries for LLM
def messages_for_user(website, resume):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt(website, resume)}
    ]

In [ ]:
def summarize_gpt(url):
    website = Website(url)
    resume = Resume('grier_resume.pdf')
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for_user(website, resume)
    )
    return response.choices[0].message.content

In [ ]:
# Uncomment correct LLM version, to ensure Jupyter has manifest ready and available prior to next part
!ollama pull llama3.2
# !ollama pull phi3:mini
# !ollama pull tinyllama

In [ ]:
payload = {
    "model": OLLAMA_MODEL,
    "headers": OLLAMA_HEADERS,
    "stream": False
}

def summarize_ollama(url):
    print(f'Be patient. Queing up {OLLAMA_MODEL}')
    website = Website(url)
    resume = Resume('grier_resume.pdf')
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=OLLAMA_MODEL,
        messages=messages_for_user(website, resume)
    )
    return response.choices[0].message.content

In [ ]:
def display_summary(url, gpt=False):
    if gpt:
        summary = summarize_gpt(url)
        display(Markdown(summary))
    else:
        # be patient
        print(summarize_ollama(url))

In [ ]:
display_summary('https://search-careers.gm.com/en/jobs/jr-202500570/data-scientist', False)